#INDIAN CASE OUTCOME PREDICTION


In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

Next we need to prepare to load a range of quantized language models, including a new 15 trillion token LLama-3 model, optimized for memory efficiency with 4-bit quantization.


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]



---



Next, we integrate LoRA adapters into our model, which allows us to efficiently update just a fraction of the model's parameters, enhancing training speed and reducing computational load.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#DATA PREP



The pre training dataset consists of **1562 indian legal cases** summarized into the following format :

<*START CASE*> \
<*FACTS*>...<*/FACTS*>\
<*ARGUMENTS*>...<*/ARGUMENTS*>\
<*OBSERVATIONS*>...<*/OBSERVATIONS*>\
<*JUDGEMENT*>...<*/JUDGEMENT*>\
<*END CASE*>

In [4]:
import re
from datasets import Dataset
from transformers import AutoTokenizer
from unsloth import is_bfloat16_supported
from tqdm import tqdm

# Load the tokenizer
EOS_TOKEN = tokenizer.eos_token

# 1. Load your raw case text file
with open("combined_cases.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 2. Extract <START CASE> ... <END CASE> blocks
case_blocks = re.findall(r"<START CASE>(.*?)<END CASE>", raw_text, re.DOTALL)

# 3. Extract structured fields from each case
def parse_case_block(block):
    def extract(tag):
        match = re.search(rf"<{tag}>(.*?)</{tag}>", block, re.DOTALL)
        return match.group(1).strip() if match else "Information not available"

    return {
        "Facts": extract("FACTS"),
        "ARGUMENTS": extract("ARGUMENTS"),
        "Observations": extract("OBSERVATIONS"),
        "JUDGMENT": extract("JUDGMENT"),
    }

# Parse all cases
parsed_cases = [parse_case_block(block) for block in tqdm(case_blocks)]

# 4. Convert to HuggingFace Dataset
dataset = Dataset.from_list(parsed_cases)

# 5. Formatting function
def formatting_prompts_func(examples):
    facts = examples["Facts"]
    arguments = examples["ARGUMENTS"]
    observations = examples["Observations"]
    judgment = examples["JUDGMENT"]

    texts = []
    for f, a, o, j in zip(facts, arguments, observations, judgment):
        text = (
            f"<Facts>\n{f}\n</Facts>\n\n"
            f"<ARGUMENTS>\n{a}\n</ARGUMENTS>\n\n"
            f"<Observations>\n{o}\n</Observations>\n\n"
            f"<JUDGMENT>\n{j}\n</JUDGMENT>{EOS_TOKEN}"
        )
        texts.append(text)

    return {"text": texts}

# 6. Apply formatting
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)


100%|██████████| 1562/1562 [00:00<00:00, 12413.09it/s]


Map:   0%|          | 0/1562 [00:00<?, ? examples/s]

# PRE-TRAINING THE MODEL

We continous pretrain the model on the case summaries by training the model on Next token prediction task to give the model legal domain knowledge

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1562 [00:00<?, ? examples/s]

In [6]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,562 | Num Epochs = 2 | Total steps = 390
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: govindarajan7272 (govindarajan7272-pes-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.655300
2,1.706200
3,1.763500
4,1.625400
5,1.692100
6,1.519900
7,1.473500
8,1.459600
9,1.512900
10,1.521500


#FINE TUNE DATASET

the Fine tuning data is of the format :

{\
  "**Instruction**": fixed prompt to predict case outcome using inputs\
  "**Inputs**": facts, observation and arguments from case summary + relevant laws from the vector db\
  "**Output**": judgement from case summary\
}

In [7]:
# this is basically the system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }
pass


In [8]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="finetune_data.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/390 [00:00<?, ? examples/s]

#FINE-TUNING MODEL

We finetune the model using LORA on our downstream task i.e. Case outcome prediction

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/390 [00:00<?, ? examples/s]

In [10]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 390 | Num Epochs = 2 | Total steps = 96
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,1.488200
2,1.367700
3,1.248800
4,1.426700
5,1.258500
6,1.120000
7,1.051500
8,1.052100
9,1.034100
10,0.862300


In [11]:
import json
from tqdm import tqdm  # optional, for a progress bar

# Load your JSON data
with open("test_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Store outputs
all_outputs = []
preds = []  # Model-generated outputs
refs = []   # Ground-truth outputs from dataset

# Loop through dataset
for i, item in enumerate(tqdm(data)):
    instruction = item["instruction"]
    input_text = item["input"]
    gold_output = item["output"]

    # Format the prompt using your alpaca-style prompt template
    prompt = alpaca_prompt.format(instruction, input_text, "")

    # Tokenize and generate
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    generated_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract only the model's response
    response_only = generated_response.split("### Response:")[-1].strip()

    # Save to combined text list (optional for inspection)
    combined_text = (
        f"### Response {i+1}\n{response_only}\n\n# Output from JSON:\n{gold_output}\n\n"
    )
    all_outputs.append(combined_text)

    # Add to evaluation lists
    preds.append(response_only)
    refs.append(gold_output)


100%|██████████| 44/44 [05:37<00:00,  7.67s/it]


In [12]:
# Combine into list of dictionaries
combined = [{"refs": ref, "preds": pred} for ref, pred in zip(refs, preds)]

# Write to JSON file
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(combined, f, indent=2, ensure_ascii=False)

#INFERENCE

In [13]:
# this is basically the system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [16]:
def query(input_text):
    FastLanguageModel.for_inference(model)  # Set the model to inference mode

    prompt = alpaca_prompt.format(
        "You are a judge. Predict the outcome of this case based on the facts, observations, judgements and relevant laws.",
        input_text,
        ""
    )

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=50,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    raw_prediction = generated_text.split(prompt)[-1].strip()
    clean_prediction = raw_prediction.split("Note:")[0].strip()

    print("\n\n=== MODEL PREDICTION ===\n")
    #print(clean_prediction)
    return clean_prediction  # This will not print again unless explicitly printed outside


In [18]:
query("""
"Facts:
The case revolves around the classification of towers and pre-fabricated buildings (PFBs) as inputs under Rule 2(k) of the CENVAT Rules for the purpose of credit benefits. The Assessee, Bharti Airtel, had claimed that these structures are inputs used in providing mobile telecommunication services.

Arguments:
The Delhi High Court and the Supreme Court upheld the judgment of Vodafone (supra) and held that towers and PFBs qualify as \"inputs\" under Rule 2(k) for the purpose of credit benefits under the CENVAT Rules. They cited various provisions, including sub-clause (ii) of Rule 2(k), which defines inputs as all goods used for providing any output service.
The Bombay High Court had a contrary view and held that towers and PFBs are not inputs but immovable property. However, this judgment was set aside by the Supreme Court.

observations:
The Supreme Court observed that Explanation-2 to Rule 2(k) is neither appropriate nor necessary in this case as sub-clause (ii) of Rule 2(k) clearly provides for inputs used for providing output services. The Court also noted that towers and PFBs are indispensable for the effective functioning of antennas, which transmit radio signals.

Relevant Laws:
Based on the provided case details, I have identified the top 5 most relevant laws that would determine the outcome of this case:

1. **Section 120A of the Indian Penal Code (IPC)**: This section defines criminal conspiracy, which is a key aspect of the case. To prove a criminal conspiracy, it must be shown that two or more people agreed to commit an offense, and at least one of them took steps to further the common object.

Relevant because: The case involves allegations of a meeting of minds between multiple individuals to achieve a common goal, which is central to the concept of criminal conspiracy under Section 120A.

2. **Section 34 of the IPC**: This section states that when a person is knowingly concerned in or party to an offense committed by another, such person may be liable for punishment as if he had committed the offense himself.

Relevant because: The case involves multiple individuals allegedly working together to achieve a common goal, which may lead to liability under Section 34. This law could be applied to determine the extent of individual responsibility in the alleged conspiracy.

3. **Section 109 of the IPC**: This section states that if a person intentionally aids another to commit an offense punishable with imprisonment for life, such person shall be punished with imprisonment for a term not less than two years and not more than seven years.

Relevant because: The case involves allegations of individuals providing aid or assistance to each other in committing a crime. Section 109 could be applied to determine the liability of those involved in providing such aid.

4. **Section 120B of the IPC**: This section states that whoever is a party to a criminal conspiracy to commit an offense punishable with death or imprisonment for life shall, if overt act follows, be punished with either description according to the nature of the offense.

Relevant because: The case involves allegations of a conspiracy to commit a serious crime. Section 120B could be applied to determine the punishment applicable to those involved in the conspiracy, depending on the severity of the planned offense.

5. **Section 34 of the Code of Criminal Procedure (CrPC)**: This section deals with the burden of proof in cases where multiple individuals are accused of a common crime.

Relevant because: The case involves allegations of multiple individuals working together to achieve a common goal, which raises questions about the distribution of the burden of proof. Section 34 of the CrPC could be applied to determine how the burden of proof should be divided among those involved in the alleged conspiracy.

These laws are relevant to determining the outcome of this case as they address the key issues of criminal conspiracy, liability, and punishment."

""")



=== MODEL PREDICTION ===



'Decision Made: The Supreme Court dismissed the Civil Appeal filed by Bharti Airtel Limited (supra) and allowed the Cross Appeals filed by M/s Indus Towers Limited (supra). In consequence, the judgment dated 19th January, 2017 passed by the Division Bench of the Bombay High Court in Tax Review Application No. 157 of 2009 was set aside, while the judgment dated 30th August, 2016 passed by the learned Single Judge of the Bombay High Court in Tax Review Application No. 155 of 2009 was restored and affirmed.'

In [19]:
query("""
"Facts:
The case involves a dispute between the appellants and the respondent-CBI regarding a contract between the parties, breach of which led to determination of the contract or underlying lease deed. The dispute centers around the coal rejects that did not contain any useful calorific value (c.v.) and were washed by the appellants at their own cost.

Arguments:
The arguments presented by the appellants include:

* The respondent-CBI embarked on a roving and fishing inquiry based on the Audit Report of the CAG, which has not attained finality.
* The underpinnings of what was a civil dispute were painted with the brush of criminality without any justification.
* The clauses of the agreements governing the parties were misinterpreted by the respondent-CBI.
* The Washability Report submitted by CIMFR, Nagpur, stated that the coal rejects did not contain any useful c.v., contradicting the assertion made by the respondent-CBI.

observations:
The court observed:

* The respondent-CBI heavily relied on the observations made in the Audit Report of the CAG to establish criminal intent against the appellants.
* The process of washing raw coal was a predominant prerequisite to meet the specified grade of coal with defined GCV for generation of power at BTPS.
* Failure to supply washed coal would have invited heavy penalties and led to serious consequences, including stoppage of power generation.

Relevant Laws:
Based on the case details provided, here are the top 5 most relevant laws that would determine the outcome of this case:

1. **Indian Penal Code (IPC) - Section 44**: This law is relevant because it deals with the definition of \"injury\" which may be caused to another person as a result of unlawful gain. The court's decision on whether the respondent was aware of the execution of fictitious sale deeds for making unlawful gains would depend on this section.

2. **Code of Criminal Procedure (CrPC) - Section 482**: This law is relevant because it deals with the power of the High Court to quash FIRs or criminal proceedings if they are found to be filed with mala fide intentions, such as harassing the respondents in this case.

3. **Indian Penal Code (IPC) - Sections related to Cheating and Dishonesty**: These laws are relevant because they deal with the offenses of cheating and dishonesty, which may have been committed by the respondent in executing fictitious sale deeds for making unlawful gains.

4. **Code of Civil Procedure (CPC) - Section 151**: This law is relevant because it deals with the power of the court to pass interim orders or make any other order to prevent abuse of process of court, which may be applicable in cases where FIRs are filed with mala fide intentions.

5. **Section 197 of the Indian Penal Code (IPC)**: This law is relevant because it deals with the offense of causing disappearance of evidence of crime, which may have been committed by the respondent in this case if they attempted to destroy or conceal evidence related to the fictitious sale deeds.

These laws are most relevant to determining the outcome of this case as they deal with the offenses committed, the power of the court to quash FIRs or criminal proceedings, and the abuse of process of court."

""")



=== MODEL PREDICTION ===



'Decision Made: The judgment delivered by the court states:\n\n* The appeals succeed, and they are allowed accordingly.\n* The impugned judgments stand set aside except for the judgment dated 18th April, 2019 passed by the learned Single Judge of the High Court of Judicature at Patna in Application u/s 96/97 CPC being Appeal No. 01JC0010 of 2020.\n* The appeal arising out of SLP (Cri) No. 10526 of 2020 stands disposed of.\n* Parties shall bear their respective costs.'

In [21]:
query("""
"Facts:
The case involves Arvind Kejriwal, the Chief Minister of Delhi, who is facing allegations under the Prevention of Money Laundering Act (PMLA). The Enforcement Directorate (ED) has filed a case against him and others. Arvind Kejriwal has been in custody for over 90 days.

Arguments:
The court argues that the reasons to believe provided by the ED are not sufficient, and therefore, no reasonable grounds exist to arrest Arvind Kejriwal under the PMLA. The court also discusses the concept of \"need and necessity to arrest\" and questions whether it refers to formal parameters or personal grounds.

observations:
The court notes that Arvind Kejriwal is an elected leader with significant importance and influence, but it does not direct him to step down or refrain from functioning as the Chief Minister. The court also observes that the questions raised in this case require in-depth consideration by a larger Bench.

Relevant Laws:
Here are the top 5 most relevant laws that could determine the outcome of this case:

1. **Section 172 of the Indian Penal Code (IPC)**: This section deals with \"Absconding to avoid service of summons or other proceeding\". It specifically addresses the offense of absconding to avoid being served with a summons, notice, or order from a public servant. Since the case involves avoiding service of a summons or order, this law is directly relevant.

2. **Section 195(1)(a)(i) of the Code of Criminal Procedure 1973 (CrPC)**: This section bars courts from taking cognizance of offenses under Sections 172 to 188 (inclusive) of the IPC without a written complaint from \"the public servant concerned\" or another public servant. Since Section 172 is involved, this law applies and may impact whether the court can take cognizance of the case.

3. **Section 173 of the Code of Criminal Procedure 1973 (CrPC)**: Although not mentioned in the provided text, Section 173 deals with police investigations and the submission of charge sheets to the courts. It is relevant because it outlines the procedures that must be followed when investigating offenses under Sections 172 to 188 of the IPC.

4. **Section 177 of the Code of Criminal Procedure 1973 (CrPC)**: This section addresses the requirements for cognizance and complaints related to public servants. Since Section 195(1)(a)(i) specifically references \"the public servant concerned\", this law may be relevant in determining the complainant's status and authority.

5. **Section 182 of the Indian Penal Code (IPC)**: This section deals with \"False information, with intent to cause public servant to use his lawful power wrongly\". Although not directly mentioned in the provided text, it is worth noting that this law addresses related offenses concerning public servants and their actions.

These laws are all relevant because they deal specifically with contempt of authority, absconding, and the requirements for cognizance and complaints regarding public servants."

""")



=== MODEL PREDICTION ===



"Decision Made: The court allows the appeal and directs Arvind Kejriwal's release on bail till the conclusion of the trial. The conditions for bail include:\n* Furnishing bail bonds for a sum of Rs.10,00,000/-;\n* Not making any public comments on the merits of the charges against him; and\n* Marking attendance before the Enforcement Directorate on every Monday and Friday between 10-11 AM.\nThe court further requests the Registry to place the matter before the Hon'ble Chief Justice of India for constitution of a Larger Bench."